# 要約 
このJupyter Notebookは、Chatbot Arenaコンペティションにおいて、チャットボットの応答の好みを予測するモデルを構築するためのものです。具体的には、与えられたプロンプトに対して2つの異なる応答のうちどちらが好まれるかを予測するタスクに取り組んでいます。

### 問題の概要
コンペティションのゴールは、ユーザーが好むチャットボットの応答を予測することであり、そのためにさまざまな大規模言語モデル（LLM）を使用して、選好のスコアを生成することが必要です。

### 使用した手法とライブラリ
1. **ライブラリのインストールとインポート**:
   - `pandas`: データ操作に使用。
   - `catboost`: 最終的なモデルの学習にはCatBoostClassifierを利用。
   - `transformers`: Hugging Faceライブラリからのモデルとトークナイザをインポート。
   - `nltk`: 自然言語処理に必要なストップワードの処理。
   - `tqdm`: プログレスバーの表示。
   - `torch`: Pytorchを使用してモデルを扱います。

2. **データ処理**:
   - 入力データの読み込みと前処理を行い、プロンプトや応答をJSON形式から辞書型に変換。
   - ストップワードリストを読み込み、不要な単語の削除を行うための準備。

3. **モデルの構築**:
   - Hugging FaceのPre-trainedモデル（RoBERTa, DeBERTa, DistilBERTなど）を用いて質問応答のパイプラインを構築。
   - それぞれのモデルに対して、データセットに基づいたスコアを計算。

4. **モデルの学習**:
   - 生成された特徴量とターゲット（勝者モデル）を使用して、CatBoostClassifierでモデルを訓練。

5. **予測と提出ファイルの作成**:
   - テストデータに対し、モデルからの予測確率を算出し、サンプル提出ファイルに追加。
   - 最終的にCSVファイルとして提出用のファイルを保存。

### 結果の出力
最終的には、予測結果を含むCSVファイルが生成されます。このノートブックはデータ処理、モデルの選定、訓練、評価、結果のフォーマットまでの一連の流れを示しており、コンペティションにおける優れたモデル構築のための実践的な手法を提供しています。

---


# 用語概説 
以下に、機械学習・深層学習の初心者がつまずきそうな専門用語を解説します。ノートブックの内容に特有のものや、実務経験がないと馴染みのない用語に焦点を当てました。

### 専門用語の解説

1. **ロジスティック回帰 (Logistic Regression)**:
   - 二項分類問題を解決するための確率モデルで、出力は0と1の間の値を取るシグモイド関数を使います。カテゴリカルデータやバイナリデータの予測に広く使用されます。

2. **CatBoostClassifier**:
   - カテゴリカルデータに特化した勾配ブースティングライブラリの一つです。他のブースティングアルゴリズムに比べて、前処理なしで直接カテゴリカルデータを扱えるのが特徴です。

3. **StratifiedKFold**:
   - クロスバリデーションの手法で、各フォールドにおけるクラスラベルの比率を元のデータと同じに保つことにより、データのバランスを取ります。クラス分布を均等に保ってモデルの評価を行うことができます。

4. **log_loss**:
   - モデルの予測確率と実際のラベルとの不一致を測るための指標で、対数損失関数とも呼ばれます。この値が小さいほど、モデルの予測が実際のラベルに近いことを示します。

5. **トークナイザー (Tokenizer)**:
   - テキストを単語やサブワードに分割するためのツールです。モデルが理解できる形式に変換するために必要で、特にNLPでは不可欠な処理です。

6. **パイプライン (Pipeline)**:
   - 一連のデータ処理やモデルのステップをまとめた手順で、トークン化、エンコーディング、予測の各段階を効率的に実行できるようにします。

7. **doc_stride**:
   - 質問応答タスクにおいて、コンテキストが長すぎる場合に、スライドさせて分割する幅を指します。これにより、モデルはより長いテキストを処理できるようになります。

8. **json.loads()**:
   - JSON（JavaScript Object Notation）形式の文字列をPythonの辞書型に変換する関数です。データの入出力やAPIとのやり取りでよく使用されます。

9. **エンコード (Encoding)**:
   - テキストデータを特定のフォーマットに変換する処理のこと。特に、文字コードの問題を解消するための修正が行われることがあります。

10. **データセット (Dataset)**:
    - 機械学習モデルに供給される訓練データや評価データのコレクションです。通常は特徴量とターゲットの情報を含みます。

11. **optuna**:
    - ハイパーパラメータの最適化を行うための自動化ライブラリです。機械学習モデルのパフォーマンスを向上させるための最適な設定を探索するために使用されます。

12. **データフレーム (DataFrame)**:
    - Pandasライブラリで提供される、構造化データを表形式で扱うためのデータ構造です。行と列に分かれており、データ操作や分析に便利です。

これらの用語を理解することで、ノートブックのコードや処理の流れがより明確になるでしょう。

---


# ライブラリ



In [ ]:
!pip install /kaggle/input/ftfy-dependeces/ftfy-6.2.0-py3-none-any.whl

In [ ]:
import pandas as pd
# ロジスティック回帰モデルをインポートするためのコードがコメントアウトされています
# from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
from ftfy import fix_encoding
import re
import nltk
from nltk.corpus import stopwords
# NLTKのストップワードをダウンロードするためのコードがコメントアウトされています
# nltk.download('stopwords')
from torch.utils.data import Dataset
import torch
import json
import optuna
import numpy as np

from tqdm.auto import tqdm
tqdm.pandas()  # プログレスバーの表示を可能にします

In [ ]:
# ストップワードリストを読み込み、リスト形式で保存します
stop_words = pd.read_csv("/kaggle/input/nltk-english-stopwords/nltk_eng_stopwords.csv")["list_of_stopwords"].tolist()

# データの読み込み



In [ ]:
# 学習データ、テストデータ、サンプル提出ファイルを読み込みます
train_df = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")
test_df = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/test.csv")
sample_df = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/sample_submission.csv")

In [ ]:
# テストデータの行数が10未満の場合、学習データを100行に制限するコードがコメントアウトされています
# if test_df.shape[0] < 10:
#     train_df = train_df[:100]

In [ ]:
def get_exploded(df: pd.DataFrame) -> pd.DataFrame:
    tmp = df.copy()
    # prmopt列、response_a列、response_b列の各値をJSON形式から辞書型に変換し、進捗バーを表示します
    tmp["prompt"] = tmp["prompt"].progress_apply(lambda x: json.loads(fix_encoding(x)))
    tmp["response_a"] = tmp["response_a"].progress_apply(lambda x: json.loads(fix_encoding(x)))
    tmp["response_b"] = tmp["response_b"].progress_apply(lambda x: json.loads(fix_encoding(x)))

    # 各列でexplodeを適用してデータを展開します
    tmp = tmp.explode(['prompt', 'response_a', 'response_b'])
    return tmp

In [ ]:
# 学習データとテストデータを展開します
tmp_train = get_exploded(train_df)
tmp_test = get_exploded(test_df)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df, col):
        self.col = col
        self.df = df.copy()
        
        # prompt列と指定した列の値をエンコード修正します
        self.df["prompt"] = self.df["prompt"].progress_apply(self.fix_encode)
        self.df[col] = self.df[col].progress_apply(self.fix_encode)
        
        # 各列でexplodeを適用してデータを展開します
        self.df = self.df.explode(['prompt', col])
    
    def fix_encode(self, x):
        return json.loads(fix_encoding(x))  # エンコードを修正します

    def __len__(self):
        return len(self.df)  # データフレームの長さを返します

    def __getitem__(self, i):
        QA_input = {}
        QA_input['question'] = str(self.df.iloc[i]["prompt"])  # 質問を取得します
        QA_input['context'] = str(self.df.iloc[i][self.col])  # コンテキストを取得します
        
        # 質問またはコンテキストが空の場合、ダミーテキストを設定します
        if not QA_input['question']:
            QA_input['question'] = 'empty_text' * 10
        if not QA_input['context']:
            QA_input['context'] = 'empty_text' * 10
        
        # 質問とコンテキストの長さを510文字に制限します
        QA_input['question'] = QA_input['question'][:510]
        QA_input['context'] = QA_input['context'][:510]

        return QA_input  # 質問とコンテキストの辞書を返します


# データセットのインスタンスを作成します
dataset_a = MyDataset(train_df, col='response_a')
dataset_b = MyDataset(train_df, col='response_b')

dataset_a_test = MyDataset(test_df, col='response_a')
dataset_b_test = MyDataset(test_df, col='response_b')

In [ ]:
# データセットの長さを確認します
len(dataset_a)

In [ ]:
# 出力結果を保存するための辞書を初期化します
outs_dict = {'dataset_a': [], 'dataset_b': []}
outs_dict_test = {'dataset_a': [], 'dataset_b': []}

# モデル

## スコアの生成



In [ ]:
# 使用するモデルのリストを定義します
model_list = [
    "deepset/roberta-base-squad2",
    "deepset/deberta-v3-base-squad2",
    "distilbert/distilbert-base-cased-distilled-squad"
#     "Palak/microsoft_deberta-large_squad"
#     'distilbert/distilbert-base-cased-distilled-squad',
#     'deepset/bert-large-uncased-whole-word-masking-squad2'
]

# Kaggleのパスを加えたモデルリストのコピーを作成します
model_list_kaggle = model_list.copy()
for i, model_name in enumerate(model_list_kaggle):
    model_list_kaggle[i] = '/kaggle/input/deberta-v3-base/' + model_name
model_list_kaggle

In [ ]:
nlp_list = []

# それぞれのモデルに対して、モデルとトークナイザーを読み込み、パイプラインを作成します
for model_name in model_list_kaggle:
    model = AutoModelForQuestionAnswering.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True)
    
    nlp = pipeline('question-answering', model=model, tokenizer=tokenizer, device='cuda', torch_dtype=torch.float16)
    
    nlp_list.append(nlp)  # パイプラインをリストに追加します

In [ ]:
# モデルのパイプラインを保存します
for model_name, pipeline in tqdm(zip(model_list, nlp_list), total=len(nlp_list)):
    pipeline.save_pretrained(model_name)

In [ ]:
def get_outs(model_list):
    outs_dict = {}
    # 各モデルに対して出力辞書を初期化します
    for model_name in model_list:
        outs_dict[f'{model_name}-a'] = []
        outs_dict[f'{model_name}-b'] = []
    return outs_dict

In [ ]:
# 出力用の辞書を作成します
outs_train = get_outs(model_list)
outs_test = get_outs(model_list)

In [ ]:
# トークナイザーの設定がコメントアウトされています
# tokenizer_kwargs = {"truncation": True, 'max_length': 512, 'padding': True}

In [ ]:
def get_score(nlp, dataset) -> list:
    scores = []
    # データセットの各サンプルに対してスコアを計算します
    for sample in tqdm(dataset, total=len(dataset)):
        try:
            out = nlp(sample, doc_stride=47)  # NLPパイプラインを使用してスコアを計算します
        except:
            print('omom')  # エラーが発生した場合の処理
            out = {}
            out['score'] = 0  # スコアを0に設定します
        scores.append(out['score'])  # スコアをリストに追加します
    return scores

In [ ]:
# 各モデルに対してトレーニングとテストのスコアを取得します
for model_name, nlp in tqdm(zip(model_list, nlp_list), total=len(model_list)):
    outs_train[f'{model_name}-a'] = get_score(nlp, dataset_a)
    outs_train[f'{model_name}-b'] = get_score(nlp, dataset_b)
    
    outs_test[f'{model_name}-a'] = get_score(nlp, dataset_a_test)
    outs_test[f'{model_name}-b'] = get_score(nlp, dataset_b_test)
    
    del nlp  # 使用が終わったモデルのパイプラインを削除します

In [ ]:
# 出力辞書をデータフレームに変換します
outs_train = pd.DataFrame(outs_train)
outs_test = pd.DataFrame(outs_test)

In [ ]:
# 学習結果を表示します
outs_train.head()

In [ ]:
# 学習結果とID、勝者モデルの情報を結合します
df_bert = pd.concat([outs_train, tmp_train[['id', 'winner_model_a', 'winner_model_b', 'winner_tie']].reset_index()], axis=1).drop('index', axis=1)
df_bert_test = pd.concat([outs_test, tmp_test['id'].reset_index()], axis=1).drop('index', axis=1)
df_bert.head()

In [ ]:
# モデルのデータセットを取得します
model_dataset = outs_train.columns

In [ ]:
# 学習データをIDでグループ化し、平均を計算します
df_bert_train = df_bert.groupby('id').mean()

df_bert_test = df_bert_test.groupby('id').mean()

In [ ]:
# 学習データとテストデータをCSVファイルに保存します
df_bert_train.to_csv('df_bert_train.csv', index=False)
df_bert_test.to_csv('df_bert_test.csv', index=False)

In [ ]:
# 学習データの最初の数行を表示します
df_bert_train.head()

## 一つの列に集約



In [ ]:
# 勝者を示す列を追加します
df_bert_train['winner'] = df_bert_train[['winner_model_a', 'winner_model_b', 'winner_tie']].apply(np.argmax, axis=1)
df_bert_train.head(2)

In [ ]:
# 不要な列を削除します
df_bert_train.drop(columns=['winner_model_a', 'winner_model_b', 'winner_tie'], inplace=True)

In [ ]:
# 変換後のデータを表示します
df_bert_train.head(2)

In [ ]:
# 最終的なデータをCSVファイルに保存します
df_bert_train.to_csv('deberts.csv', index=False)

## モデルのトレーニング



In [ ]:
# CatBoostClassifierのモデルを定義します
model = CatBoostClassifier(verbose=False, random_state=2024)

In [ ]:
target = 'winner'  # 予測するターゲット列を定義します

In [ ]:
# 学習データの最初の数行を表示します
df_bert_train.head()

In [ ]:
# 特徴量とターゲットを分割します
X_train = df_bert_train.drop(columns=target)  # 特徴量
y_train = df_bert_train[target]  # ターゲット

In [ ]:
# モデルを学習させます
model.fit(X_train, y_train)

In [ ]:
# テストデータに対して予測を行います
y_pred = model.predict_proba(df_bert_test)  # 予測確率を取得します
y_pred

# 提出用ファイルの作成



In [ ]:
# サンプル提出ファイルに予測結果を追加します
sample_df[['winner_model_a', 'winner_model_b', 'winner_tie']] = y_pred

In [ ]:
# 提出ファイルの内容を表示します
sample_df

In [ ]:
# 提出ファイルをCSV形式で保存します
sample_df.to_csv('submission.csv', index=False)